# Pandas Analysis

In [1]:
import pandas as pd
import numpy as np

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

## Aggregation and Grouping
Here we will use the Planets dataset, available via the Seaborn package. It gives information on planets that astronomers have discovered around other stars (known as extrasolar planets or exoplanets for short). It can be downloaded with a simple Seaborn command.

In [ ]:
import seaborn as sns
planets = sns.load_dataset('planets')
planets.head()

Earlier, we explored some of the data aggregations available for NumPy ```arrays```. As with a one-dimensional NumPy ```array```, for a Pandas ```Series``` the aggregates return a single value.

In [ ]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.rand(5))
print("Series sum:", ser.sum())
print("Series mean:", ser.mean())

For a ```DataFrame```, by default the aggregates return results within each column. By specifying the ```axis``` argument, you can instead aggregate within each row.

In [ ]:
df = pd.DataFrame({'A': rng.rand(5),
                   'B': rng.rand(5)})
print("Default aggregation by column for mean:\n", df.mean())
print("Aggregation by row for mean:\n", df.mean(axis="columns")) # could also use axis=1


In addition to standard aggregation functions, Pandas includes a convenience function called ```describe()``` that computes several common aggregates for each column and returns the result. Let's take a look at how it works using the planets dataset and dropping rows with missing values.

In [ ]:
planets.dropna().describe()

This can be a useful way to begin understanding the overall properties of a dataset. For example, we see in the ```year``` column that although exoplanets were discovered as far back as 1989, half of all known expolanets were not discovered until 2010 or after. This is largely thanks to the Kepler mission, which is a space-based telescope specifically designed for finding eclipsing planets around other stars.

To go deeper into the data, however, simple aggregates are often not enough. The next level of data summarization is the ```groupby``` operation, which allows you to quickly and efficiently compute aggregates on subsets of data.

## GroupBy: Split, Apply, Combine
Simple aggregations can give you a flavor of your dataset, but often we would prefer to aggregate conditionally on some label or index: this is implemented in the so-called groupby operation. The name "group by" comes from a command in the SQL database language, but it is perhaps more illuminative to think of it in the terms first coined by the ```R``` developer Hadley Wickham: split, apply, combine.

![](figures/03.08-split-apply-combine.png)

This makes clear what the ```groupby``` accomplishes:
- The *split* step involves breaking up and grouping a ```DataFrame``` depending on the value of the specified key.
- The *apply* step involves computing some function, usually an aggregate, transformation, or filtering, within the individual groups.
- The *combine* step merges the results of these operations into an output array.

While this could certainly be done manually using some combination of the masking, aggregation, and merging commands covered earlier, an important realization is that the intermediate splits do not need to be explicitly instantiated. Rather, the ```GroupBy``` can (often) do this in a single pass over the data, updating the sum, mean, count, min, or other aggregate for each group along the way. The power of the ``GroupBy`` is that it abstracts away these steps: the user need not think about how the computation is done under the hood, but rather thinks about the operation as a whole.

As a concrete example, let's take a look at using Pandas for the computation shown in this diagram using the planets dataset.

In [ ]:
planets.groupby('method')

Notice that what is returned is not a set of ```DataFrames```, but a ```DataFrameGroupBy``` object. This object is where the magic is: you can think of it as a special view of the ```DataFrame```, which does no actual computation until the aggregation is applied. This "lazy evaluation" approach means that common aggregates can be implemented very efficiently in a way that is almost transparent to the user.The ```GroupBy``` object supports column indexing in the same way as a ```DataFrame```, and returns a modified GroupBy object. The ```GroupBy``` object supports column indexing in the same way as a ```DataFrame```, and returns a modified ```GroupBy``` object.

In [ ]:
planets.groupby('method')['orbital_period'].median()

### Aggregate, fileter, transform, apply
The preceding discussion focused on aggregation for the combine operation, but there are more options available. In particular, ```GroupBy``` objects have ```aggregate()```, ```filter()```, ```transform()```, and ```apply()``` methods that efficiently implement a variety of useful operations before combining the grouped data.

In [ ]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': rng.randint(0, 10, 6)},
                   columns = ['key', 'data1', 'data2'])
df

#### Aggregation
We're now familiar with ```GroupBy``` aggregations with ```sum()```, ```median()```, etc., but the aggregate() method allows for even more flexibility. It can take a string, a function, or a list thereof, and compute all the aggregates at once.

In [ ]:
df.groupby('key').aggregate(['min', np.median, max])

Another useful pattern is to pass a dictionary mapping column names to operations to be applied on that column.

In [ ]:
df.groupby('key').aggregate({'data1': 'min',
                             'data2': 'max'})

#### Filtering
A filtering operation allows you to drop data based on the group properties. For example, we might want to keep all groups in which the standard deviation is larger than some critical value (e.g., 4 for column ```data2```).

In [ ]:
def filter_func(x):
    return x['data2'].std() > 4

display('df', "df.groupby('key').std()", "df.groupby('key').filter(filter_func)") # Drops rows where key = A based on std value

#### Transformation
While aggregation must return a reduced version of the data, transformation can return some transformed version of the full data to recombine. For such a transformation, the output is the same shape as the input. A common example is to center the data by subtracting the group-wise mean.

In [ ]:
df.groupby('key').transform(lambda x: x - x.mean()) # A lambda function is a Pythonic way to quickly specify a function in-place

#### Apply
The ```apply()``` method lets you apply an arbitrary function to the group results. The function should take a ```DataFrame```, and return either a Pandas object (e.g., ```DataFrame```, ```Series```) or a scalar; the combine operation will be tailored to the type of output returned.

For example, below is an ```apply()``` that normalizes the first column by the sum of the second.

In [ ]:
def norm_by_data2(x):
    # x is a DataFrame of group values
    x['data1'] /= x['data2'].sum()
    return x

display('df', "df.groupby('key').apply(norm_by_data2)") # Groupby key and apply summation of data2 column sum for each group

### Specifying the split key
In the simple examples presented before, we split the ```DataFrame``` on a single column name. This is just one of many options by which the groups can be defined, and we'll go through some other options for group specification here.

##### A list, series, or index as the grouping keys
The key can be any ```series```, ```list```, or ```index``` with a length matching that of the ```DataFrame```.

In [ ]:
L = [0, 1, 0, 1, 2, 0]
display('df', 'df.groupby(L).sum()')

#### A dictionary or series mapping index to group
Another method is to provide a ```dictionary``` that maps index values to the group keys.

In [ ]:
df2 = df.set_index('key')
mapping = {'A': 'vowel', 'B': 'consonant', 'C': 'consonant'}
display('df2', 'df2.groupby(mapping).sum()')

It is also possible to combine methods to form a multi-index grouping.

In [ ]:
df2.groupby(['key', mapping]).mean()

Below is a more complex example that combines several Python operations (some we've covered to this point and others that are noted in comments). This shows the power of combining many of the operations we've discussed up to this point when looking at realistic datasets. We immediately gain a coarse understanding of when and how planets have been discovered over the past several decades!

In [ ]:
decade = 10 * (planets['year'] // 10) # // is floor division that rounds down to the nearest integer
decade = decade.astype(str) + 's' # Changes the integer decade into a string and adds 's' to the end of each decade: e.g., 1980s
decade.name = 'decade' # Gives the Series a name 'decade'
planets.groupby(['method', decade])['number'].sum().unstack().fillna(0) # Group planets DataFrame by method and decade Series

## Pivot tables
We have seen how the ```GroupBy``` abstraction lets us explore relationships within a dataset. A *pivot table* is a similar operation that is commonly seen in spreadsheets and other programs that operate on tabular data. The *pivot table* takes simple column-wise data as input, and groups the entries into a two-dimensional table that provides a multidimensional summary of the data. A *pivot table* is essentially a multidimensional version of ```GroupBy``` aggregation. That is, you split-apply-combine, but both the split and the combine happen across a two-dimensional grid  rather than a one-dimensional index.

For the examples in this section, we'll use the database of passengers on the Titanic available through the Seaborn package.

In [ ]:
titanic = sns.load_dataset('titanic')
titanic.head()

### Manual pivot using Groupby

It is possible to use ```Groupby``` to perform pivot table operations, but it can be cumbersome. To start learning more about this data, we might begin by grouping according to gender, survival status, or some combination thereof. Let's look at survival rate by gender using ```Groupby``` operations. This immediately gives us some insight: overall, three of every four females on board survived, while only one in five males survived!

In [ ]:
titanic.groupby('sex')[['survived']].mean()

This is useful, but we might like to go one step deeper and look at survival by both sex and, say, class. Using the vocabulary of ```GroupBy```, we might proceed using something like this: we group by class and gender, select survival, apply a mean aggregate, combine the resulting groups, and then unstack the hierarchical index to reveal the hidden multidimensionality.

In [ ]:
titanic.groupby(['sex', 'class'])['survived'].aggregate('mean').unstack()

### Pivot using built-in pivot_table

Two-dimensional ```GroupBy``` is common enough that Pandas includes a convenience routine, ```pivot_table```, which succinctly handles this type of multi-dimensional aggregation.

In [ ]:
titanic.pivot_table('survived', index='sex', columns='class') # default aggfunc is mean

In [ ]:
titanic.pivot_table('survived', index='sex', columns='class', aggfunc='std') # specify a different aggfunc 'std'

### Multi-level pivot tables
Just as in the ```GroupBy```, the grouping in pivot tables can be specified with multiple levels, and via a number of options. For example, we might be interested in looking at age as a third dimension. We'll bin the age using the ```pd.cut``` function. Further, we can apply the same strategy when working with the columns as well; let's add info on the fare paid using pd.qcut to automatically compute quantiles.

In [ ]:
age = pd.cut(titanic['age'], [0, 18, 80])
fare = pd.qcut(titanic['fare'], 2)
titanic.pivot_table('survived', index=['sex', age], columns=[fare, 'class'])

### Additional pivot table options
The full call signature of the ```pivot_table``` method of ```DataFrames``` is as follows:
```
# call signature as of Pandas 0.18
DataFrame.pivot_table(data, values=None, index=None, columns=None,
                      aggfunc='mean', fill_value=None, margins=False,
                      dropna=True, margins_name='All')
```
Two of the options, ```fill_value``` and ```dropna```, have to do with missing data and are fairly straightforward; we will not show examples of them here.

The ```aggfunc``` keyword controls what type of aggregation is applied, which is a mean by default. As in the GroupBy, the aggregation specification can be a string representing one of several common choices (e.g., ```'sum'```, ```'mean'```, ```'count'```, ```'min'```, ```'max'```, etc.) or a function that implements an aggregation (e.g., ```np.sum()```, ```min()```, ```sum()```, etc.). Additionally, it can be specified as a dictionary mapping a column to any of the above desired options.

In [ ]:
titanic.pivot_table(index='sex', columns='class',
                    aggfunc={'survived':'sum', 'fare':'mean'})

In [ ]:
titanic

At times it's useful to compute totals along each grouping. This can be done via the margins keyword.

In [ ]:
titanic.pivot_table('survived', index='sex', columns='class', margins=True)

## References
https://jakevdp.github.io/PythonDataScienceHandbook/